In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atherosclerosis"
cohort = "GSE125771"

# Input paths
in_trait_dir = "../../input/GEO/Atherosclerosis"
in_cohort_dir = "../../input/GEO/Atherosclerosis/GSE125771"

# Output paths
out_data_file = "../../output/preprocess/Atherosclerosis/GSE125771.csv"
out_gene_data_file = "../../output/preprocess/Atherosclerosis/gene_data/GSE125771.csv"
out_clinical_data_file = "../../output/preprocess/Atherosclerosis/clinical_data/GSE125771.csv"
json_path = "../../output/preprocess/Atherosclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"RNA expression data from calcified human carotid atherosclerotic plaques"
!Series_summary	"Although unstable atherosclerosis in the carotid bifurcation is a significant etiology behind ischemic stroke, clinical imaging methods to distinguish stable from vulnerable lesions are lacking and selection of patients for stroke-preventive intervention still relies on surrogate variables with moderate predictive power, such as the degree of luminal narrowing. Here we combined clinical and diagnostic imaging information by comuted tomography to select patients with calcified plaques for large scale molecular analysis, in an effort to increase our understanding of the pathophysiology behind carotid plaque instability as related to patient- and plaque- phenotype."
!Series_overall_design	"Patients undergoing surgery for high-grade (>50% NASCET) carotid stenosis at the Department of Vascular Surgery, Karolinska University Hospital, Stockholm, Sweden were consec

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains RNA expression data from carotid atherosclerotic plaques
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the Sample Characteristics Dictionary:

# 2.1 Trait Availability
# All samples are from carotid atherosclerotic plaques
# Since we're studying Atherosclerosis and all samples are atherosclerotic plaques,
# we can use this as a binary trait where all samples have the condition
trait_row = 0  # All samples are atherosclerotic plaques

# Function to convert trait values
def convert_trait(value):
    # All samples have atherosclerosis (carotid-atherosclerotic-plaque)
    if 'atherosclerotic' in value.lower():
        return 1  # Has atherosclerosis
    return None  # Unknown or not clear

# 2.2 Age Availability
# Age information is available in row 3
age_row = 3

# Function to convert age values
def convert_age(value):
    try:
        # Extract the number after "age: "
        if ':' in value:
            age_str = value.split(':', 1)[1].strip()
            return float(age_str)  # Convert to float for continuous value
        return None
    except:
        return None

# 2.3 Gender Availability
# Gender information is available in row 2
gender_row = 2

# Function to convert gender values
def convert_gender(value):
    if ':' in value:
        gender_str = value.split(':', 1)[1].strip().lower()
        if 'female' in gender_str:
            return 0  # Female
        elif 'male' in gender_str:
            return 1  # Male
    return None  # Unknown or unclear

# 3. Save Metadata - Initial filtering
# Determine if trait data is available
is_trait_available = trait_row is not None
# Use the validate_and_save_cohort_info function for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Use the geo_select_clinical_features function
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save the clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM3581706': [1.0, 73.0, 1.0], 'GSM3581707': [1.0, 60.0, 1.0], 'GSM3581708': [1.0, 81.0, 0.0], 'GSM3581709': [1.0, 85.0, 1.0], 'GSM3581710': [1.0, 60.0, 0.0], 'GSM3581711': [1.0, 84.0, 0.0], 'GSM3581712': [1.0, 76.0, 0.0], 'GSM3581713': [1.0, 57.0, 1.0], 'GSM3581714': [1.0, 71.0, 1.0], 'GSM3581715': [1.0, 69.0, 1.0], 'GSM3581716': [1.0, 79.0, 0.0], 'GSM3581717': [1.0, 78.0, 1.0], 'GSM3581718': [1.0, 79.0, 1.0], 'GSM3581719': [1.0, 54.0, 1.0], 'GSM3581720': [1.0, 72.0, 1.0], 'GSM3581721': [1.0, 73.0, 1.0], 'GSM3581722': [1.0, 64.0, 1.0], 'GSM3581723': [1.0, 67.0, 1.0], 'GSM3581724': [1.0, 63.0, 0.0], 'GSM3581725': [1.0, 75.0, 0.0], 'GSM3581726': [1.0, 62.0, 1.0], 'GSM3581727': [1.0, 64.0, 1.0], 'GSM3581728': [1.0, 73.0, 1.0], 'GSM3581729': [1.0, 81.0, 1.0], 'GSM3581730': [1.0, 79.0, 1.0], 'GSM3581731': [1.0, 72.0, 1.0], 'GSM3581732': [1.0, 71.0, 1.0], 'GSM3581733': [1.0, 75.0, 1.0], 'GSM3581734': [1.0, 74.0, 1.0], 'GSM3581735': [1.0, 76.0, 0.0], 'GSM35817

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atherosclerosis/GSE125771/GSE125771_series_matrix.txt.gz
Gene data shape: (65535, 40)
First 20 gene/probe identifiers:
Index(['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1',
       'TC01000004.hg.1', 'TC01000005.hg.1', 'TC01000006.hg.1',
       'TC01000007.hg.1', 'TC01000008.hg.1', 'TC01000009.hg.1',
       'TC01000010.hg.1', 'TC01000011.hg.1', 'TC01000012.hg.1',
       'TC01000013.hg.1', 'TC01000014.hg.1', 'TC01000015.hg.1',
       'TC01000016.hg.1', 'TC01000017.hg.1', 'TC01000018.hg.1',
       'TC01000019.hg.1', 'TC01000020.hg.1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers follow the format "TC01000001.hg.1" which appears to be probe IDs from a microarray platform
# (likely Affymetrix or similar), not standard human gene symbols.
# These are technical identifiers that need to be mapped to standard gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Based on the preview, 'gene_assignment' column appears to contain gene symbol information
# The format seems to be: "Accession // Gene Symbol // Description // Location // ID"
# Let's create a mapping function to extract gene symbols from this column

def extract_gene_symbols(gene_assignment_str):
    """Extract gene symbols from gene_assignment column values"""
    if not isinstance(gene_assignment_str, str) or gene_assignment_str == '---':
        return []
    
    # Split by multiple gene entries (separated by ///)
    gene_entries = gene_assignment_str.split('///')
    gene_symbols = []
    
    for entry in gene_entries:
        parts = entry.strip().split('//')
        if len(parts) >= 2:  # We need at least accession and gene symbol
            symbol = parts[1].strip()
            if symbol and symbol != '---' and symbol != 'NULL':
                gene_symbols.append(symbol)
    
    return gene_symbols

# Create a mapping dataframe with probe IDs and gene symbols
print("\nCreating mapping between probe IDs and gene symbols...")
mapping_data = gene_annotation[['ID', 'gene_assignment']].copy()
mapping_data = mapping_data.dropna(subset=['gene_assignment'])
mapping_data = mapping_data[mapping_data['gene_assignment'] != '---']

# Extract all gene symbols from the gene_assignment column
mapping_data['Gene'] = mapping_data['gene_assignment'].apply(extract_gene_symbols)

# Filter out rows with empty gene lists and display sample of mapping data
mapping_data = mapping_data[mapping_data['Gene'].apply(len) > 0]
print(f"Mapping data shape: {mapping_data.shape}")
print("Sample mapping data:")
print(preview_df(mapping_data[['ID', 'Gene']], n=5))

# Count how many probes map to at least one gene symbol
genes_mapped = mapping_data.shape[0]
total_probes = gene_annotation.shape[0]
mapping_percentage = (genes_mapped / total_probes) * 100 if total_probes > 0 else 0
print(f"\nFound gene symbols for {genes_mapped} out of {total_probes} probes ({mapping_percentage:.2f}%)")

# Save the mapping for later use if needed
gene_mapping = mapping_data[['ID', 'Gene']]



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'category', 'locus type', 'notes', 'SPOT_ID']
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His)

Mapping data shape: (32670, 3)
Sample mapping data:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'Gene': [['DDX11L1', 'DDX11L5', 'DDX11L1'], ['MIR1302-11', 'MIR1302-10', 'MIR1302-9', 'MIR1302-2', 'MIR1302-11', 'MIR1302-10', 'MIR1302-9', 'MIR1302-2', 'MIR1302-11', 'MIR1302-10', 'MIR1302-9', 'MIR1302-2', 'OTTHUMG00000000959', 'RP11-34P13.3', 'OTTHUMG00000000959', 'RP11-34P13.3'], ['OR4F5', 'OR4F5', 'OR4F5'], ['OTTHUMG00000002525', 'RP11-34P13.9'], ['LOC100132287', 'LOC100133331', 'LOC101060495', 'LOC101060494', 'LOC101059936', 'LOC100996502', 'LOC100996328', 'LOC100287894', 'LOC100132062', 'OTTHUMG00000156968', 'RP4-669L17.10', 'OTTHUMG00000156968', 'RP4-669L17.10', 'OTTHUMG00000156968', 'RP4-669L17.10', 'OTTHUMG00000156968', 'RP4-669L17.10']]}

Found gene symbols for 32670 out of 2692193 probes (1.21%)


### Step 6: Gene Identifier Mapping

In [7]:
# Function for a simplified version of apply_gene_mapping that handles the case more directly
def simplified_gene_mapping(expression_df, mapping_df):
    """
    Direct implementation of probe-to-gene mapping that preserves more data
    """
    # Prepare a dictionary to collect expression values by gene
    gene_to_expr = {}
    
    # Process each probe in the expression data
    for probe_id in expression_df.index:
        # Skip if probe not in mapping
        matching_rows = mapping_df[mapping_df['ID'] == probe_id]
        if matching_rows.empty:
            continue
            
        # Get the gene list for this probe
        gene_list = []
        for genes in matching_rows['Gene'].values:
            if isinstance(genes, list) and genes:
                gene_list.extend(genes)
        
        # Skip if no genes mapped
        if not gene_list:
            continue
            
        # Get unique gene names (to avoid double-counting)
        unique_genes = list(set(gene_list))
        if not unique_genes:
            continue
            
        # For each sample, distribute expression value among genes
        expr_values = expression_df.loc[probe_id]
        contribution = 1.0 / len(unique_genes)  # Equal distribution
        
        # Add contribution to each gene
        for gene in unique_genes:
            if gene not in gene_to_expr:
                gene_to_expr[gene] = pd.Series(0, index=expression_df.columns)
            gene_to_expr[gene] += expr_values * contribution
    
    # Convert the dictionary to a DataFrame
    if gene_to_expr:
        return pd.DataFrame(gene_to_expr).T
    else:
        return pd.DataFrame(index=[], columns=expression_df.columns)

# Let's try our simplified mapping function
print("Attempting simplified gene mapping approach...")
# Ensure gene_mapping contains lists in the Gene column
if isinstance(gene_mapping['Gene'].iloc[0], str):
    gene_mapping['Gene'] = gene_mapping['Gene'].apply(lambda x: [x] if isinstance(x, str) else x)

gene_data_mapped = simplified_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after simplified mapping: {gene_data_mapped.shape}")

# If we still don't have data, attempt with a more aggressive extraction approach
if gene_data_mapped.shape[0] == 0:
    print("Simplified mapping failed. Using a more aggressive approach...")
    
    # Extract a direct mapping dictionary by parsing the gene_assignment column
    direct_mapping_dict = {}
    
    # Function to extract the most likely gene name from a gene assignment string
    def extract_main_gene(text):
        if not isinstance(text, str) or text == '---':
            return None
        
        # Try to find a standard gene name pattern with one or more capital letters followed by digits/letters
        patterns = [
            r'\/\/\s([A-Z][A-Z0-9]+)\s\/\/',  # Pattern: // GENE //
            r'\/\/\s([A-Za-z][A-Za-z0-9]+)\s\/\/',  # More permissive pattern
            r'\/\/\s([A-Z][A-Z0-9\-]+)\s'  # Another common pattern
        ]
        
        for pattern in patterns:
            matches = re.findall(pattern, text)
            if matches:
                return matches[0]
        
        # Fallback to any word that looks like a gene symbol
        words = text.split()
        for word in words:
            if re.match(r'^[A-Z][A-Z0-9]{1,14}$', word) and word not in ['RNA', 'DNA', 'PCR']:
                return word
                
        return None
    
    # Process each probe in the expression data
    for probe_id in gene_data.index:
        # Find the matching annotation
        annotation = gene_annotation[gene_annotation['ID'] == probe_id]
        if not annotation.empty and 'gene_assignment' in annotation.columns:
            gene_assignment = annotation['gene_assignment'].values[0]
            gene = extract_main_gene(gene_assignment)
            if gene:
                direct_mapping_dict[probe_id] = gene
    
    print(f"Created direct mapping dictionary with {len(direct_mapping_dict)} entries")
    
    # Apply the direct mapping
    gene_sums = {}
    for probe_id, gene in direct_mapping_dict.items():
        if gene not in gene_sums:
            gene_sums[gene] = pd.Series(0, index=gene_data.columns)
        gene_sums[gene] += gene_data.loc[probe_id]
    
    if gene_sums:
        gene_data_mapped = pd.DataFrame(gene_sums).T
        print(f"Gene expression data shape after direct mapping: {gene_data_mapped.shape}")
    else:
        gene_data_mapped = pd.DataFrame(index=[], columns=gene_data.columns)
        print("Failed to extract any gene data with all approaches.")

# Use the result (either from simplified_gene_mapping or direct mapping)
gene_data = gene_data_mapped

# Preview the first few rows of the gene expression data
print("Gene expression data preview (first 5 genes):")
print(preview_df(gene_data, n=5))

# Save the gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

Attempting simplified gene mapping approach...


Gene expression data shape after simplified mapping: (40679, 40)
Gene expression data preview (first 5 genes):
{'GSM3581706': [6.373333333333333, 6.373333333333333, 3.4191666666666665, 0.6783333333333333, 3.4191666666666665], 'GSM3581707': [4.125, 4.125, 3.655, 0.735, 3.655], 'GSM3581708': [3.6849999999999996, 3.6849999999999996, 3.4908333333333332, 0.7216666666666667, 3.4908333333333332], 'GSM3581709': [3.523333333333333, 3.523333333333333, 3.8724999999999996, 0.7183333333333333, 3.8724999999999996], 'GSM3581710': [3.33, 3.33, 3.575, 0.7216666666666667, 3.575], 'GSM3581711': [3.7416666666666667, 3.7416666666666667, 3.63, 0.7116666666666666, 3.63], 'GSM3581712': [3.793333333333333, 3.793333333333333, 3.5075, 0.7416666666666667, 3.5075], 'GSM3581713': [3.6983333333333333, 3.6983333333333333, 3.5108333333333333, 0.7483333333333333, 3.5108333333333333], 'GSM3581714': [3.475, 3.475, 3.560833333333333, 0.7183333333333333, 3.560833333333333], 'GSM3581715': [3.5633333333333335, 3.563333333333

Gene expression data saved to ../../output/preprocess/Atherosclerosis/gene_data/GSE125771.csv
